In [ ]:
!pip install facebook-business

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.8/914.8 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 36.7 MB/s eta 0:00:00
  Created wheel for curlify: filename=curlify-2.2.1-py3-none-any.whl size=2666 sha256=05c56e82e06c334c4d905acbabe481a7cc3a5c55f7edf5b476142e5917ca05ee
  Stored in directory: /root/.cache/pip/wheels/01/e8/2f/e0e986ab1e8956d5ad10aabc059767c822d6114ed295a28dd5
Successfully built curlify


In [ ]:
'''

from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.business import Business
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adset import AdSet
from facebook_business.adobjects.campaign import Campaign
import pandas as pd
# Initialize the Facebook Ads API
FacebookAdsApi.init(access_token='')
# The Business ID for your parent account
business_id = '770208824098437'
business = Business(business_id)
# Retrieve all ad accounts under the parent business account
child_accounts = business.get_owned_ad_accounts()
# Fetch insights for each child account
all_insights = []
# Fields to retrieve from each ad set
fields = [
    'name',
    'age_min',
    'age_max',
    'geo_locations',
    'lifetime_budget',
]
exclude_ad_account = '571601977885778'
for account in child_accounts:
    if account['id'] == f'act_{exclude_ad_account}':
        continue
    adsets = AdAccount(account['id']).get_ad_sets(fields=fields)
    for adset in adsets:
        # Extract age range
        age_min = adset.get('age_min', '')
        age_max = adset.get('age_max', '')
        age_range = f"{age_min}-{age_max}" if age_min and age_max else ''
        # Extract location
        location = adset.get('geo_locations', {}).get('countries', [])
        # Extract budget
        budget = adset.get('lifetime_budget', '')
        # Combine the data into a structured format
        all_insights.append({
            'ad_set_name': adset.get('name', ''),
            'age_range': age_range,
            'location': location,
            'budget': budget,
        })
# Convert to DataFrame and export
data = pd.DataFrame(all_insights)
data.to_csv('./adset_insights_new.csv', index=False)

'''

## Extract 1 Single Media object

In [ ]:
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.instagramuser import InstagramUser
import pandas as pd

# Initialize the Facebook Ads API
FacebookAdsApi.init(access_token='')

# Instagram user ID for the media
ig_media_id = '18422344687057320'

# Fields to retrieve from the Instagram media
fields = [
    'id',
    'name',
    'caption',
    'ig_id',
    'username',
]

# Function to fetch Instagram media
def fetch_ig_media(ig_media_id, fields):
    ig_media = InstagramUser(ig_media_id)
    return ig_media.api_get(fields=fields)

# Fetch Instagram media data
ig_media_data = fetch_ig_media(ig_media_id, fields)

# Convert to DataFrame and export
data = pd.DataFrame([ig_media_data])
data.to_csv('./ig_media_data.csv', index=False)


## Check for Single page id , How many instagram business account & Media objects it have

In [ ]:
import requests

# Step 3: Get a User Access Token
user_access_token = ""

# Step 4: Get the User's Pages
def get_page_id(user_access_token):
    url = f"https://graph.facebook.com/v19.0/me/accounts?access_token={user_access_token}"
    response = requests.get(url)
    page_data = response.json()["data"]
    page_id = page_data[0]["id"]  # Assuming the first page, you may need to implement logic to choose the correct one
    print('page_id',len(page_id),page_id)
    return page_id

# Step 5: Get the Page's Instagram Business Account
def get_instagram_business_account(page_id, user_access_token):
    url = f"https://graph.facebook.com/v19.0/{page_id}?fields=instagram_business_account&access_token={user_access_token}"
    response = requests.get(url)
    instagram_business_account_id = response.json()["instagram_business_account"]["id"]
    print('instagram_business_account_id',len(instagram_business_account_id),instagram_business_account_id)
    return instagram_business_account_id

# Step 6: Get the Instagram Business Account's Media Objects
def get_media_objects(instagram_business_account_id, user_access_token):
    url = f"https://graph.facebook.com/v19.0/{instagram_business_account_id}/media?access_token={user_access_token}"
    response = requests.get(url)
    media_objects = response.json()["data"]
    print('media_objects',len(media_objects),media_objects
    return media_objects

# Main function to execute the steps
def main():
    page_id = get_page_id(user_access_token)
    instagram_business_account_id = get_instagram_business_account(page_id, user_access_token)
    media_objects = get_media_objects(instagram_business_account_id, user_access_token)
    print("Media Objects:")
    for media_object in media_objects:
        print(media_object)

if __name__ == "__main__":
    main()


## Collect all the Media object in a dataframe

In [ ]:
import requests
import pandas as pd

# Step 1: Get all page IDs
def get_page_ids(user_access_token):
    url = f"https://graph.facebook.com/v19.0/me/accounts?access_token={user_access_token}"
    response = requests.get(url)
    page_data = response.json()["data"]
    page_ids = [page["id"] for page in page_data]
    print('page_ids',len(page_ids), page_ids)
    print("*"*10)
    return page_ids

# Step 2: Get the list of Page's Instagram Business Account IDs
def get_instagram_business_accounts(page_ids, user_access_token):
    instagram_business_account_ids = []
    for page_id in page_ids:
        url = f"https://graph.facebook.com/v19.0/{page_id}?fields=instagram_business_account&access_token={user_access_token}"
        response = requests.get(url)
        instagram_business_account_id = response.json().get("instagram_business_account", {}).get("id")
        if instagram_business_account_id:
            instagram_business_account_ids.append(instagram_business_account_id)
        else:
            instagram_business_account_ids.append("None")
    print('instagram_business_account_ids',len(instagram_business_account_ids), instagram_business_account_ids)
    print("*"*10)
    return instagram_business_account_ids

# Step 3: Get the list of Instagram Business Account's Media Objects
def get_media_objects(instagram_business_account_ids, user_access_token):
    all_media_objects = []
    for instagram_business_account_id in instagram_business_account_ids:
        url = f"https://graph.facebook.com/v19.0/{instagram_business_account_id}/media?access_token={user_access_token}"
        response = requests.get(url)
        media_objects = response.json().get("data", [])
        all_media_objects.extend(media_objects)
    print('all_media_objects',len(all_media_objects), all_media_objects)
    print("*"*10)
    return all_media_objects

# Step 4: Fetch media objects and return as a DataFrame
def fetch_media_objects(user_access_token):
    page_ids = get_page_ids(user_access_token)
    instagram_business_account_ids = get_instagram_business_accounts(page_ids, user_access_token)
    media_objects = get_media_objects(instagram_business_account_ids, user_access_token)

    # Extract ig_media_id from each media object
    ig_media_ids = [media_object['id'] for media_object in media_objects]

    # Create a DataFrame with the ig_media_id column
    df = pd.DataFrame({'ig_media_id': ig_media_ids})

    return df

In [ ]:
# Example usage
if __name__ == "__main__":
    user_access_token = "EAAKPAmfjLb0BOZCAkF3kopjyaKZB7jyXOUFYEO1BoQHAqM9MYAIQxHFab9ilkrYdDf2ppM1UqdHNTMbUgjstVZCbUs2ZCUCur6Cf4T6h7mHKzEwdX7HNqOp4NOUoJZCD0PhAg5CR31jg0HkI82By2F8shL2svQfVu77MckbiT7vAKkPkqZAWZAZCfl915rledFd3"
    df=fetch_media_objects(user_access_token)
    df.to_csv('data.csv')

page_ids 25 ['190782224126008', '211093108755572', '109273095586133', '105157759309877', '325180077954930', '119616017795191', '100409929737524', '106324389135557', '335387863208380', '102281672803589', '105652045791484', '106008905744351', '110224435301470', '1359404084108195', '105991065633617', '106439142252945', '109668491918957', '102490727873254', '106301565561288', '112295654857056', '102321399285761', '103496589167949', '105408552195054', '101087898940076', '107547118532752']
**********
instagram_business_account_ids 25 ['None', 'None', '17841459322931489', '17841459623400260', '17841401178066431', '17841460152010891', '17841459278765706', 'None', '17841401600284786', 'None', '17841458411655298', 'None', '17841457264027502', '17841402725409815', '17841456393659954', 'None', '17841412071145366', 'None', '17841453667948617', '17841453882145590', '17841454141019051', 'None', 'None', '17841449134540478', '17841456601370309']
**********
all_media_objects 347 [{'id': '184223446870573

page_ids ['190782224126008', '211093108755572', '109273095586133', '105157759309877', '325180077954930', '119616017795191', '100409929737524', '106324389135557', '335387863208380', '102281672803589', '105652045791484', '106008905744351', '110224435301470', '1359404084108195', '105991065633617', '106439142252945', '109668491918957', '102490727873254', '106301565561288', '112295654857056', '102321399285761', '103496589167949', '105408552195054', '101087898940076', '107547118532752']



instagram_business_account_ids ['17841459322931489', '17841459623400260', '17841401178066431', '17841460152010891', '17841459278765706', '17841401600284786', '17841458411655298', '17841457264027502', '17841402725409815', '17841456393659954', '17841412071145366', '17841453667948617', '17841453882145590', '17841454141019051', '17841449134540478', '17841456601370309']




all_media_objects [{'id': '18422344687057320'}, {'id': '18341513119105223'}, {'id': '18041321242696009'}, {'id': '17954587292760397'}, {'id': '18012456521196212'}, {'id': '17861332218102892'}, {'id': '18418728811017854'}, {'id': '17879533455024993'}, {'id': '17867185890070727'}, {'id': '17914059089903243'}, {'id': '18013733528008556'}, {'id': '17880036522034668'}, {'id': '18000627298959594'}, {'id': '17981145521362881'}, {'id': '18010902703763338'}, {'id': '18028372483516374'}, {'id': '18190312561266815'}, {'id': '17864212775900521'}, {'id': '18178281286277066'}, {'id': '18367280947012177'}, {'id': '17871483635914261'}, {'id': '18023432392908706'}, {'id': '18424490344013447'}, {'id': '17894285573989842'}, {'id': '18123745183340857'}, {'id': '17898022610967350'}, {'id': '18421550512015004'}, {'id': '18036579970759774'}, {'id': '17971016837679458'}, {'id': '18098855128379731'}, {'id': '18001501958422403'}, {'id': '18023452633961501'}, {'id': '18036704650749417'}, {'id': '18094762864396523'}, {'id': '18015395684124479'}, {'id': '17887621037938152'}, {'id': '18048814138595394'}, {'id': '17998797614395047'}, {'id': '18416847430009878'}, {'id': '17887570775995070'}, {'id': '18014429297501381'}, {'id': '18022095211958524'}, {'id': '17846340150154784'}, {'id': '18415078792024190'}, {'id': '18141138403312817'}, {'id': '18029787031860741'}, {'id': '18051273652411991'}, {'id': '18008354971765966'}, {'id': '17978306048346750'}, {'id': '17992665479043632'}, {'id': '17972719745388281'}, {'id': '17943663419655235'}, {'id': '17958284144479394'}, {'id': '18286279282139225'}, {'id': '17988924998041200'}, {'id': '17970466592234675'}, {'id': '17896697228810191'}, {'id': '17955873800459152'}, {'id': '17889893096828939'}, {'id': '18033658843493380'}, {'id': '17929905404691872'}, {'id': '17929778189687182'}, {'id': '18037052188843094'}, {'id': '18046414993658929'}, {'id': '18093730030411776'}, {'id': '17992513559357249'}, {'id': '18048167965613079'}, {'id': '18097326679387338'}, {'id': '18300415855197197'}, {'id': '17996347553221002'}, {'id': '17973247307644518'}, {'id': '17933092928687009'}, {'id': '18074808982427344'}, {'id': '18017051980801947'}, {'id': '17883476441968925'}, {'id': '18038969563596314'}, {'id': '18022702852834342'}, {'id': '17888768867939786'}, {'id': '17860146495045182'}, {'id': '18078376447405848'}, {'id': '18038237503567845'}, {'id': '18025351579705456'}, {'id': '17863555887020614'}, {'id': '17977979039563557'}, {'id': '18001877507130878'}, {'id': '18031100983615286'}, {'id': '17928540293779500'}, {'id': '18021960512498537'}, {'id': '18233356261253109'}, {'id': '17999010131334969'}, {'id': '18010381553370233'}, {'id': '17951916623661018'}, {'id': '18318861715122483'}, {'id': '18313596538181776'}, {'id': '17898957809974323'}, {'id': '18122228032343114'}, {'id': '18004168355434846'}, {'id': '17985512738635197'}, {'id': '17954775773758987'}, {'id': '17849611557144582'}, {'id': '18044991994642993'}, {'id': '18422335945063930'}, {'id': '17987579738477167'}, {'id': '18035834095849369'}, {'id': '17972807180545558'}, {'id': '17971434200689962'}, {'id': '18127690870327279'}, {'id': '17861914293037527'}, {'id': '18073794556433082'}, {'id': '18037530373616392'}, {'id': '18299232790122930'}, {'id': '18270514117162039'}, {'id': '17904179600959944'}, {'id': '18011892860363824'}, {'id': '18000454169605623'}, {'id': '18016524758334830'}, {'id': '18016908815268006'}, {'id': '18002690810449045'}, {'id': '18025220900104402'}, {'id': '18019495334492535'}, {'id': '17898755288977639'}, {'id': '17860612431116119'}, {'id': '18045498118660514'}, {'id': '18250363753220021'}, {'id': '18427567042056604'}, {'id': '18181190929289398'}, {'id': '18034387891809891'}, {'id': '17966116238702084'}, {'id': '18409237555057703'}, {'id': '17986466318440043'}, {'id': '17909490149912175'}, {'id': '18040673485657002'}, {'id': '18024811894781343'}, {'id': '18076411546447807'}, {'id': '18049584724561169'}, {'id': '17971671071668080'}, {'id': '17939742530784669'}, {'id': '17872584459015282'}, {'id': '17843202627188241'}, {'id': '18006049439358365'}, {'id': '17899347056945609'}, {'id': '18045465910610532'}, {'id': '18047796070578216'}, {'id': '18295018081198371'}, {'id': '18102819232371759'}, {'id': '18017210317950989'}, {'id': '18010871327064471'}, {'id': '17982192896540185'}, {'id': '18029874094644747'}, {'id': '18042939421544702'}, {'id': '18005677250032039'}, {'id': '18389194669035492'}, {'id': '18281300707156616'}, {'id': '18267006733084269'}, {'id': '17916871985736409'}, {'id': '18392264881048005'}, {'id': '18022682809698751'}, {'id': '18013364533769702'}, {'id': '18000461849141060'}, {'id': '17951985779585597'}, {'id': '18265096921161722'}, {'id': '17844169566083405'}, {'id': '18034952704857822'}, {'id': '17978181260352546'}, {'id': '18064699936327137'}, {'id': '17995960078475531'}, {'id': '17911467593425068'}, {'id': '17908869248438821'}, {'id': '17967055852543174'}, {'id': '18154501870215651'}, {'id': '17877938975632644'}, {'id': '17894964548318284'}, {'id': '17877983399291942'}, {'id': '17852970959555393'}, {'id': '18082118830265698'}, {'id': '18151217914089388'}, {'id': '17868150323382695'}, {'id': '18079967152267175'}, {'id': '17868014471257289'}, {'id': '17845056825203302'}, {'id': '17949373322788051'}, {'id': '18004853741226811'}, {'id': '18027955321985625'}, {'id': '17967178142709273'}, {'id': '18006051956192391'}, {'id': '18009511493461290'}, {'id': '18024682847045198'}, {'id': '18003627788406174'}, {'id': '18260377438210864'}, {'id': '17913481841829953'}, {'id': '17875387218011847'}, {'id': '17989364333447729'}, {'id': '17978386868659644'}, {'id': '18077934430458476'}, {'id': '18017828245965356'}, {'id': '17868717327039322'}, {'id': '18012656036056759'}, {'id': '17977489811661187'}, {'id': '18049423807540065'}, {'id': '18100199485380147'}, {'id': '17912603372912462'}, {'id': '18050342578587629'}, {'id': '18027334300928182'}, {'id': '17878799844024194'}, {'id': '18037433974721950'}, {'id': '18023319313900752'}, {'id': '17985391469641281'}, {'id': '18127429273329135'}, {'id': '18426189061046428'}, {'id': '17910959966825544'}, {'id': '18070861006470118'}, {'id': '18046747696623958'}, {'id': '18413026396052495'}, {'id': '17874067017047514'}, {'id': '17912060957878439'}, {'id': '18019176800072459'}, {'id': '18018878105061264'}, {'id': '18007114280470226'}, {'id': '17993450831529863'}, {'id': '18020571289833100'}, {'id': '18023304763768725'}, {'id': '17883890831992071'}, {'id': '18116788663332933'}, {'id': '18028362484697328'}, {'id': '17947977002723470'}, {'id': '17986528487529629'}, {'id': '18018483166779606'}, {'id': '18006219577977498'}, {'id': '17937315272765131'}, {'id': '17879667452978914'}, {'id': '18022967665887305'}, {'id': '18016849649173838'}, {'id': '18026148649966256'}, {'id': '18014013551275277'}, {'id': '18095840830402522'}, {'id': '18136360369317736'}, {'id': '17986481006447660'}, {'id': '17933652782802587'}, {'id': '18041428819718271'}, {'id': '18020904406987463'}, {'id': '17946198191684584'}, {'id': '17912123462893670'}, {'id': '18023664016534126'}, {'id': '18023456782512217'}, {'id': '17996294896749981'}, {'id': '18000951814711887'}, {'id': '17970927371187010'}, {'id': '17929207199760878'}, {'id': '17903290511959417'}, {'id': '18013870613261895'}, {'id': '18407476627070235'}, {'id': '18015786989329313'}, {'id': '18032457991920205'}, {'id': '17999556578310750'}, {'id': '17950481213657241'}, {'id': '18220375330284605'}, {'id': '18312234238125382'}, {'id': '18063437896519608'}, {'id': '18043634812629711'}, {'id': '17903558057858909'}, {'id': '18029948980744060'}, {'id': '18298739539158317'}, {'id': '17961659264604021'}, {'id': '18015691124278751'}, {'id': '18316575082192025'}, {'id': '18068608072486865'}, {'id': '18019595909080252'}, {'id': '18013107956172587'}, {'id': '17882030415007502'}, {'id': '18029478328854560'}, {'id': '18047185036602522'}, {'id': '17856874428078715'}, {'id': '18024968180048391'}, {'id': '18006621599212218'}, {'id': '18041857300691512'}, {'id': '17934820406739033'}, {'id': '18031991245939177'}, {'id': '18005610032437768'}, {'id': '17990692805631579'}, {'id': '18030267913821500'}, {'id': '18074882380474401'}, {'id': '18002487092260111'}, {'id': '17937306812814293'}, {'id': '18058700371536625'}, {'id': '18044660647646564'}, {'id': '17992110446627264'}, {'id': '17989685918631230'}, {'id': '18013854941048516'}, {'id': '18022228181092543'}, {'id': '17929530749836016'}, {'id': '17936486588719808'}, {'id': '18038294266818031'}, {'id': '18006908753518131'}, {'id': '18016440950274722'}, {'id': '17948121131664605'}, {'id': '17966923175701817'}, {'id': '18019174522909626'}, {'id': '18235173385248800'}, {'id': '18104213566383355'}, {'id': '18053329645598157'}, {'id': '17842328589200901'}, {'id': '18008850467152220'}, {'id': '18135020368324422'}, {'id': '18053813515574005'}, {'id': '18012570668243799'}, {'id': '18033643609854602'}, {'id': '17931191594740155'}, {'id': '17997449651575952'}, {'id': '17990244509392628'}, {'id': '18028290913777865'}, {'id': '18013025786475552'}, {'id': '18089055631417057'}, {'id': '17915396228863973'}, {'id': '18380272924073474'}, {'id': '17865759744033306'}, {'id': '18017517290094551'}, {'id': '17997675311229695'}, {'id': '18047225173594178'}, {'id': '18214307824285754'}, {'id': '17997403562387366'}, {'id': '18205944715285204'}, {'id': '17917323392841598'}, {'id': '18004215053261801'}, {'id': '17893522940929017'}, {'id': '18113494345359551'}, {'id': '18283325737087080'}, {'id': '18004807880077635'}, {'id': '18003160523337587'}, {'id': '17974907123641859'}, {'id': '18022331002757004'}, {'id': '18042470149599344'}, {'id': '18136033252309750'}, {'id': '17978807018392780'}, {'id': '17884932035900514'}, {'id': '17957318297563148'}, {'id': '18068313148448070'}, {'id': '18031294813712339'}, {'id': '18026396479691098'}, {'id': '17980275797533864'}, {'id': '18030044803633951'}, {'id': '17935173800761819'}, {'id': '18186471268268460'}, {'id': '17980753025286603'}, {'id': '17996825956860447'}, {'id': '17974738664485482'}, {'id': '17986344052970312'}, {'id': '18006522937633522'}]


In [ ]:
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.instagramuser import InstagramUser
import pandas as pd

# Initialize the Facebook Ads API
FacebookAdsApi.init(access_token='')

# Function to fetch Instagram media
def fetch_ig_media(ig_media_id, fields):
    ig_media = InstagramUser(ig_media_id)
    return ig_media.api_get(fields=fields)

# Read 'ig_media_id' from data.csv
data = pd.read_csv('data.csv')
ig_media_ids = data['ig_media_id'].tolist()

# Define columns for the final DataFrame
columns = ['ig_media_id', 'id', 'caption', 'ig_id', 'username']
final_data = []

# Iterate over each ig_media_id
for ig_media_id in ig_media_ids:
    # Fetch Instagram media data
    ig_media_data = fetch_ig_media(ig_media_id, fields=columns[1:])

    # Append to final data
    final_data.append({'ig_media_id': ig_media_id, **ig_media_data})

# Convert final data to DataFrame
final_df = pd.DataFrame(final_data, columns=columns)

# Export DataFrame to CSV
final_df.to_csv('instagram_posts.csv', index=False)


/usr/local/lib/python3.10/dist-packages/facebook_business/utils/api_utils.py:16: UserWarning:  does not allow field caption
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/facebook_business/utils/api_utils.py:16: UserWarning:  does not allow field ig_id
  warnings.warn(message)
